In [1]:
import os
file_dir = r".\llm-20-questions"
discuss_path = file_dir + r"\discuss.json"
kernels_path = file_dir + r"\kernels.json"
output_dir = file_dir + "_jp"

In [2]:
#jsonファイルを読み込む
import json

with open(discuss_path, "r", encoding="utf-8") as f:
    discuss_info = json.load(f)

with open(kernels_path, "r", encoding="utf-8") as f:
    kernels_info = json.load(f)

In [3]:
#discuss_infoを"votes"順に並び替え
discuss_info_sort_by_vite = sorted(discuss_info, key=lambda x: x["votes"], reverse=True)

#discuss_infoを"comments"順に並び替え
discuss_info_sort_by_comments = sorted(discuss_info, key=lambda x: x["comments"], reverse=True)

#kernel_infoを"totalVotes  "順に並び替え
kernels_info_sort_by_totalVotes = sorted(kernels_info, key=lambda x: x["totalVotes  "], reverse=True)

In [4]:
import re
#リンク一覧
discuss_link_sort_by_vite = []
discuss_link_sort_by_comments = []
kernels_directory_sort_by_totalVotes = []

for discuss in discuss_info_sort_by_vite:
  file_name = discuss['title']
  # エスケープシーケンスを削除
  file_name = re.sub(r'\\x[0-9a-fA-F]{2}', '', file_name)

  # 無効な文字を置換または除去
  file_name = re.sub(r'[\.=<>:"/\\|?*]', '', file_name)
  file_name = file_name.replace(" ", "_")
  if len(file_name) > 60:
      file_name = file_name[:59]+"…"
  file_name = file_name + ".md"
  link = "./discuss/" + file_name
  discuss_link_sort_by_vite.append(link)

for discuss in discuss_info_sort_by_comments:
  file_name = discuss['title']
  # エスケープシーケンスを削除
  file_name = re.sub(r'\\x[0-9a-fA-F]{2}', '', file_name)

  # 無効な文字を置換または除去
  file_name = re.sub(r'[\.=<>:"/\\|?*]', '', file_name)
  file_name = file_name.replace(" ", "_")
  if len(file_name) > 60:
      file_name = file_name[:59]+"…"
  file_name = file_name + ".md"
  link = "./discuss/" + file_name
  discuss_link_sort_by_comments.append(link)

for kernel in kernels_info_sort_by_totalVotes:
  directory = kernel['title']
  # エスケープシーケンスを削除
  directory = re.sub(r'\\x[0-9a-fA-F]{2}', '', directory)

  # 無効なファイル名の文字を定義
  invalid_chars = r'<>:"/\|?*'

  # 無効な文字を置換または除去
  directory = re.sub(r'[\.=<>:"/\\|?*]', '', directory)
  directory = directory.replace(" ", "_")
  if len(directory) > 60:
      directory = directory[:59]+"…"

  kernels_directory_sort_by_totalVotes.append(directory)

In [5]:
#各kernels_directory_sort_by_totalVotesの中に一つipynbファイルがあるので、そのファイル名を取得

import os
kernels_ipynb_sort_by_totalVotes = []
for directory in kernels_directory_sort_by_totalVotes:
  for file in os.listdir(file_dir + r"\kernels" + "\\" + directory):
    if file.endswith(".ipynb"):
      kernels_ipynb_sort_by_totalVotes.append(file)
      break
#kernels_directory_sort_by_totalVotesとkernels_ipynb_sort_by_totalVotesをzipして、リンクを作成
kernels_link_sort_by_totalVotes = []
for directory, ipynb in zip(kernels_directory_sort_by_totalVotes, kernels_ipynb_sort_by_totalVotes):
  link = "./kernels/" + directory + "/" + ipynb
  kernels_link_sort_by_totalVotes.append(link)

In [6]:
def append_suffix_to_filename(file_path, suffix="_with_summary"):
    # ファイルパスを分解
    directory, filename = os.path.split(file_path)
    # 拡張子を分解
    name, ext = os.path.splitext(filename)
    # 新しいファイル名を作成
    new_filename = f"{name}{suffix}{ext}"
    # 新しいファイルパスを作成
    new_file_path = os.path.join(directory, new_filename)
    return new_file_path

In [7]:
#全てのディスカッションを結合した一つのマークダウンファイルを作成
import dis

for i,link in enumerate(discuss_link_sort_by_vite):
  #ファイルを読み込む
  with open(file_dir + discuss_link_sort_by_vite[i][1:], "r", encoding="utf-8") as f:
    origin_discuss = f.read()
  with open(output_dir + discuss_link_sort_by_vite[i][1:], "r", encoding="utf-8") as f:
    transrated_discuss = f.read()
  with open(output_dir + append_suffix_to_filename(discuss_link_sort_by_vite[i][1:]), "r", encoding="utf-8") as f:
    transrated_discuss_with_summary = f.read()
  discuss_summary = transrated_discuss_with_summary.replace(transrated_discuss, "")
  text = discuss_summary + "\n\n"
  text += f"""<style>
.column-left{{
  float: left;
  width: 47.5%;
  text-align: left;
}}
.column-right{{
  float: right;
  width: 47.5%;
  text-align: left;
}}
.column-one{{
  float: left;
  width: 100%;
  text-align: left;
}}
</style>


<div class="column-left">

# original

{origin_discuss}

</div>
<div class="column-right">

# 日本語訳

{transrated_discuss}

</div>"""
  #output_dir +"/comparison" + discuss_link_sort_by_vite[i][1:]のディレクトリを作成
  os.makedirs(os.path.dirname(output_dir +"/comparison" + discuss_link_sort_by_vite[i][1:]), exist_ok=True)
  with open(output_dir +"/comparison" + discuss_link_sort_by_vite[i][1:], "w", encoding="utf-8") as f:
    f.write(text)



In [8]:
template_markdown =  """<style>
.column-left{{
  float: left;
  width: 47.5%;
  text-align: left;
}}
.column-right{{
  float: right;
  width: 47.5%;
  text-align: left;
}}
.column-one{{
  float: left;
  width: 100%;
  text-align: left;
}}
</style>


<div class="column-left">

# original

{origin_source}

</div>
<div class="column-right">

# 日本語訳

{transrated_source}

</div>"""

template_code = """<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{{
  float: left;
  width: 47.5%;
  text-align: left;
}}
.column-right{{
  float: right;
  width: 47.5%;
  text-align: left;
}}
.column-one{{
  float: left;
  width: 100%;
  text-align: left;
}}
</style>


<div class="column-left">

# original

```python
{origin_source}
```

</div>
<div class="column-right">

# 日本語訳

```python
{transrated_source}
```

</div>
</details>"""

In [9]:
import nbformat

#全てのディスカッションを結合した一つのマークダウンファイルを作成
for i,link in enumerate(kernels_link_sort_by_totalVotes):
  new_cells = []
  #ファイルを読み込む
  with open(file_dir + kernels_link_sort_by_totalVotes[i][1:], "r", encoding="utf-8") as f:
    notebook = nbformat.read(f, as_version=4)
    origin_cells = [cell for cell in notebook.cells if cell.cell_type == "markdown" or cell.cell_type == "code"]
  with open(output_dir + kernels_link_sort_by_totalVotes[i][1:], "r", encoding="utf-8") as f:
    notebook = nbformat.read(f, as_version=4)
    transrated_cells = [cell for cell in notebook.cells if cell.cell_type == "markdown" or cell.cell_type == "code"]
    new_cells.append(transrated_cells[0])
    new_cells.append(transrated_cells[1])
    transrated_cells = transrated_cells[2:]
    for origin_cell, transrated_cell in zip(origin_cells, transrated_cells):
      if origin_cell.cell_type == "markdown":
        new_cells.append(nbformat.v4.new_markdown_cell(template_markdown.format(origin_source=origin_cell.source, transrated_source=transrated_cell.source)))
      elif origin_cell.cell_type == "code":
        new_cells.append(nbformat.v4.new_markdown_cell(template_code.format(origin_source=origin_cell.source, transrated_source=transrated_cell.source)))
        new_cells.append(transrated_cell)
    notebook.cells = new_cells
  #output_dir +"/comparison" + link[1:]のディレクトリを作成
  os.makedirs(os.path.dirname(output_dir +"/comparison" + link[1:]), exist_ok=True)
  with open(output_dir +"/comparison" + link[1:], "w", encoding="utf-8") as f:
    nbformat.write(notebook, f)
    